In [ ]:
!pip install pandas requests lxml tqdm

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from utils import *
import json

In [ ]:
with open("./config.json", "r") as fp:
    config = json.load(fp)

In [ ]:
DATA_DIR_PATH = config["data_dir_path"]

# Read SNLI datasets and generate UD tree and concat to df

In [ ]:
df_snli_train = pd.read_csv(os.path.join(DATA_DIR_PATH, "snli_train.tsv"), delimiter='\t', index_col=0)
df_snli_dev = pd.read_csv(os.path.join(DATA_DIR_PATH, "snli_dev.tsv"), delimiter='\t', index_col=0)
df_snli_test = pd.read_csv(os.path.join(DATA_DIR_PATH, "snli_test.tsv"), delimiter='\t', index_col=0)

In [ ]:
len(df_snli_train), len(df_snli_dev), len(df_snli_test)

In [ ]:
df_snli_train_head = df_snli_train.head()

In [ ]:
def concat_udtree_to_df(df):
    udtree1_list = []
    udtree2_list = []
    ud1_list = []
    ud2_list = []
    
    sample_list = list(df.itertuples())
    for i, sample in enumerate(tqdm(sample_list)):
            if "ud1" in df:
                ud1 = json.loads(sample.ud1)
            else:
                ud1 = udify_predict(sample.sentence1)
            
            if "ud2" in df:
                ud2 = json.loads(sample.ud2)
            else:
                ud2 = udify_predict(sample.sentence2)
            
            udtree1 = ET.tostring(ud2tree(ud1, sample.sentence1), encoding="utf-8").decode()
            udtree2 = ET.tostring(ud2tree(ud2, sample.sentence2), encoding="utf-8").decode()
            
            ud1 = json.dumps(ud1)
            ud2 = json.dumps(ud2)
            
            udtree1_list.append(udtree1)
            udtree2_list.append(udtree2)
            ud1_list.append(ud1)
            ud2_list.append(ud2)

    df["udtree1"] = udtree1_list
    df["udtree2"] = udtree2_list
    if not "ud1" in df:
        df["ud1"] = ud1_list
    
    if not "ud2" in df:
        df["ud2"] = ud2_list
    
    return df

In [ ]:
df_snli_train_head = concat_udtree_to_df(df_snli_train_head)
df_snli_train_head

In [ ]:
df_snli_dev = concat_udtree_to_df(df_snli_dev)
df_snli_dev.head()

In [ ]:
df_snli_dev.to_csv(os.path.join(DATA_DIR_PATH, "snli_dev.tsv"), sep="\t")

In [ ]:
df_snli_test = concat_udtree_to_df(df_snli_test)
df_snli_test.head()

In [ ]:
df_snli_test.to_csv(os.path.join(DATA_DIR_PATH, "snli_test.tsv"), sep="\t")